## Write into database and get Data from API

Dieses Notebook hat das Ziel, die benötigten Daten zur Vorhersage des ID3-Preises der türkischen Strombörse über die API der Strombörse herunterzuladen. Die Daten stammen von der Website der Börse: https://www.epias.com.tr/en/. Der Prozess besteht darin, die Daten in das Notebook zu laden und anschließend in eine SQL-Datenbank zu schreiben, um in verschiedenen Notebooks darauf zugreifen zu können.

In [ ]:
# Verbindung zu Drive herstellen:
# google.colab.drive wird verwendet, um Google Drive zu mounten.
from google.colab import drive
# sqlite3 wird für den Zugriff und die Manipulation der SQLite-Datenbank verwendet.
import sqlite3
# datetime wird genutzt, um mit Datums- und Zeitangaben zu arbeiten.
from datetime import datetime
# requests wird verwendet, um HTTP-Anfragen zu senden und Daten von der API abzurufen.
import requests
# pandas wird verwendet, um die abgerufenen Daten zu analysieren und zu manipulieren.
import pandas as pd
# Mount Google Drive: Hiermit wird Google Drive in das Colab-Notebook eingebunden. Dies ist nützlich, um Daten persistent zu speichern, da der Speicherplatz des Colab-Notebooks temporär ist und beim Beenden der Sitzung verloren geht.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# please execute this code to check if the folder structure is correct. We will use links to address these folders and when they are not there, the code won't work.

import os

def create_folder_structure(base_path):
    # Definierte Ordnerstruktur
    folder_structure = [
        "01_Data_AI_SS24",
        "01_Data_AI_SS24/Project_Turkisch_Energy_Market",
        "01_Data_AI_SS24/Project_Turkisch_Energy_Market/self_created_python_packages",
        "01_Data_AI_SS24/Project_Turkisch_Energy_Market/ExcelBestModels",
        "01_Data_AI_SS24/Project_Turkisch_Energy_Market/databases",
        "01_Data_AI_SS24/Project_Turkisch_Energy_Market/DecisionTreeRules",
        "01_Data_AI_SS24/Project_Turkisch_Energy_Market/DecisionTreeRules/RandomForest",
        "01_Data_AI_SS24/Project_Turkisch_Energy_Market/DecisionTreeRules/RandomForest/Regression",
        "01_Data_AI_SS24/Project_Turkisch_Energy_Market/DecisionTreeRules/RandomForest/Classification",
        "01_Data_AI_SS24/Project_Turkisch_Energy_Market/DecisionTreeRules/BoostedTree",
        "01_Data_AI_SS24/Project_Turkisch_Energy_Market/DecisionTreeRules/BoostedTree/Regression",
        "01_Data_AI_SS24/Project_Turkisch_Energy_Market/DecisionTreeRules/BoostedTree/Classification"
        "01_Data_AI_SS24/Project_Turkisch_Energy_Market/PresentationData",
        "01_Data_AI_SS24/Project_Turkisch_Energy_Market/TradingProfitExcel",
    ]

    # Überprüfe und erstelle die Ordnerstruktur
    for folder in folder_structure:
        folder_path = os.path.join(base_path, folder)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
            print(f"Erstellt: {folder_path}")
        else:
            # Check if there is a naming conflict in the specified level
            folder_name = os.path.basename(folder_path)
            parent_path = os.path.dirname(folder_path)
            if any(f != folder_name and os.path.isdir(os.path.join(parent_path, f)) for f in os.listdir(parent_path)):
                print(f"Ordner mit ähnlichem Namen existiert bereits in {parent_path}. Bitte benennen Sie den Ordner um.")
                return

    print("Ordnerstruktur ist vollständig und korrekt. Bitte trenne die Laufzeit undverbinde dich erneut, damit das Notebook die Ordenr laden kann.")

# Basispfad für Google Drive
base_path = '/content/drive/My Drive'

# Erstelle die Ordnerstruktur
create_folder_structure(base_path)


In [ ]:
def connect_to_database(name_of_database, path = "/content/drive/MyDrive/01_Data_AI_SS24/Project_Turkish_Energy_Market/databases"):
  """
  Funktion zum erstellen oder connecten einer Datenbank in dem dafür vorgesehenen Ordner
  """
  # Verbindung zur Datenbank herstellen
  conn = sqlite3.connect(path + "/" + name_of_database)

  # Cursor-Objekt erstellen
  cursor = conn.cursor()

  return conn, cursor

def write_dataframe_to_database(df, table_name, connection, cursor, show_succ_msg = True):
    """
    Funktion zum schreiben in die Datenbank. Diese Funktion erzeugt eine neue Tabelle / schreibt in eine bestehende Tabelle einen DataFrame aus python.
    """
    # Erstelle die Tabelle in der Datenbank
    create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(df.columns)})"
    cursor.execute(create_table_query)

    # Füge die Daten aus dem DataFrame in die Tabelle ein
    insert_query = f"INSERT INTO {table_name} VALUES ({', '.join(['?'] * len(df.columns))})"
    cursor.executemany(insert_query, df.values.tolist())

    # Speichere die Änderungen
    connection.commit()

    if show_succ_msg:
      	print(f'{table_name} Table now in database, database not closed, please close later')

Festlegen der Start- und Enddaten unserer benötigten Daten. Falls diese Daten geändert werden, müssen alle anderen Notebooks erneut ausgeführt werden.

In [ ]:
url_first_ = "https://seffaflik.epias.com.tr/electricity-service"


startDate_ = "2021-01-01T00:00:00+03:00"
endDate_ = "2023-12-31T00:00:00+03:00"
body_request_ = {
    "startDate": startDate_,
    "endDate": endDate_,
}

startDate_formatted_ = startDate_[:10].replace("-","_")
endDate_formatted_ = endDate_[:10].replace("-","_")

In [ ]:
def get_data_from_api(url_sec, body_request, url_first = "https://seffaflik.epias.com.tr/electricity-service"):
  """
  Funktion um Verbindung zur API herzustellen und einen DataFrame als Antwort zu bekommen.
  """

  url_main = url_first + url_sec

  try:
    response = requests.post(url_main, json=body_request)

    if response.status_code != 200:
      print(response.status_code)
      print(response.text)

    # JSON-Daten aus der Antwort extrahieren
    data = response.json()

    # JSON-Daten in einen DataFrame konvertieren
    df = pd.DataFrame(data['items'])

  except requests.exceptions.RequestException as e:
      print("Error fetching data:", e)

  return df.columns, df

In [ ]:
def get_DataFrame_from_SQL_database(cursor, table_name, startDate_formatted = startDate_formatted_, endDate_formatted = endDate_formatted_, limit = 0):
  """
  Funktion um aus der SQL Datenbank eine Tabelle als DataFrame zu bekommen mit dem man in python arbeiten kann.
  """
  query = f'SELECT * FROM {table_name}_{startDate_formatted}_{endDate_formatted}'

  if limit > 0:
    query = query + f' LIMIT {limit}'

  cursor.execute(query)
  rows = cursor.fetchall()
  columns = [column[0] for column in cursor.description]

  return pd.DataFrame(rows, columns=columns)

# Scraping Data

Diese Verbindungen tauchen in den Notebooks öfter auf. Hieran kann erkannt werden, auf welche der Datenbanken gerade zugegriffen wird. Im folgenden werden dann die API Daten geladen und in dei Datenbank geschrieben.

In [ ]:
# create database to store original data
[conn_, cur_] = connect_to_database('original_data.db')

## Volume weighted average price

Spalten sind: Date, Hour, Weighted Avergae Price (MWh)


In [ ]:
url_2_weighted_average_prices_ = "/v1/markets/idm/data/weighted-average-price"

[df_weighted_averages_columns_, df_weighted_average_prices_] = get_data_from_api(url_2_weighted_average_prices_, body_request_)

write_dataframe_to_database(df_weighted_average_prices_, table_name = f'WEIGHTED_AVERAGE_PRICES_{startDate_formatted_}_{endDate_formatted_}', connection = conn_, cursor = cur_)

# delete dataframe and columns
del df_weighted_averages_columns_, df_weighted_average_prices_

WEIGHTED_AVERAGE_PRICES_2021_01_01_2023_12_31 Table now in database, database not closed, please close later


## Day Ahead

**Day Ahead Preise (Market Clearing Prices)**

Spalten sind: Date, Hour, MCP (TL/MWh), MCP (USD/MWh), MCP (EUR/MWh)

In [ ]:
url_2_day_ahead_prices_ = "/v1/markets/dam/data/mcp"

[df_day_ahead_prices_columns_, df_day_ahead_prices_] = get_data_from_api(url_2_day_ahead_prices_, body_request_)

write_dataframe_to_database(df_day_ahead_prices_, table_name = f'DAY_AHEAD_PRICES_{startDate_formatted_}_{endDate_formatted_}', connection = conn_, cursor = cur_)

# delete Dataframe and columns
del df_day_ahead_prices_columns_, df_day_ahead_prices_

DAY_AHEAD_PRICES_2021_01_01_2023_12_31 Table now in database, database not closed, please close later


**Day Ahead Trade Volumen**

Spalten sind: Date, Hour, Trade Volume (TL)

In [ ]:
url_2_day_ahead_volume_ = "/v1/markets/dam/data/day-ahead-market-trade-volume"

[df_day_ahead_volume_columns_, df_day_ahead_volume_] = get_data_from_api(url_2_day_ahead_volume_, body_request_)

write_dataframe_to_database(df_day_ahead_volume_, table_name = f'DAY_AHEAD_VOLUME_{startDate_formatted_}_{endDate_formatted_}', connection = conn_, cursor = cur_)

# delete Dataframe and columns
del df_day_ahead_volume_columns_, df_day_ahead_volume_

DAY_AHEAD_VOLUME_2021_01_01_2023_12_31 Table now in database, database not closed, please close later


## Production

**Geplante Produktion**

Spalten sind: Date, Hour,
Total (MWh),
Natural Gas,
Wind,
Lignite,
Black Coal,
Import Coal,
Fuel Oil,
Geothermal,
Dam,
Naphtha,
Biomass,
River,
Other

In [ ]:
# Hierfür brauch man noch die Regionen, weil die produktion anscheinend nur regional angeboten wird, es gibt aber nur eine region
url_2_regions_ = "/v1/generation/data/region-list"

url_regio_ = url_first_ + url_2_regions_
# muss mit einem get auseführt werden
response_ = requests.get(url_regio_)

response_.json()

{'items': [{'regionId': 1, 'regionShortName': 'TR1'}]}

In [ ]:
url_2_planned_production_ = "/v1/generation/data/dpp"

body_request_region_ = {
    "startDate": startDate_,
    "endDate": endDate_,
    "region": "TR1",

}

[df_planned_production_columns_, df_planned_production_] = get_data_from_api(url_2_planned_production_, body_request_region_)

write_dataframe_to_database(df_planned_production_, table_name = f'PLANNED_PRODUCTION_{startDate_formatted_}_{endDate_formatted_}', connection = conn_, cursor = cur_)

# delete Dataframe and columns
del df_planned_production_columns_, df_planned_production_

PLANNED_PRODUCTION_2021_01_01_2023_12_31 Table now in database, database not closed, please close later


**Tatsächliche Einspeisung**

Spalten sind: Date, Hour,
Total (MWh),
Natural Gas,
Dam,
Lignite,
River,
Import Coal,
Solar,
Wind,
Fuel Oil,
Geothermal,
Asphaltite Coal,
Black Coal,
Biomass,
Naphtha,
LNG,
International Import,
International Export,
Other

In [ ]:
# problem: man kann nur maximal ein jahr nehmen, deswegen nehme 3 mal 1 Jahr

In [ ]:


url_2_injection_quantitiy_ = "/v1/generation/data/injection-quantity"

startDate_datetime_ = datetime.strptime(startDate_, "%Y-%m-%dT%H:%M:%S%z")
endDate_datetime_ = datetime.strptime(endDate_, "%Y-%m-%dT%H:%M:%S%z")

current_date = startDate_datetime_

while current_date.year <= endDate_datetime_.year:
    next_year = current_date.replace(year=current_date.year + 1)
    new_start_date = current_date.strftime("%Y-%m-%dT%H:%M:%S%z")
    new_end_date = next_year.strftime("%Y-%m-%dT%H:%M:%S%z")

    new_start_date = new_start_date[:-2] + ":" + new_start_date[-2:]
    new_end_date = new_end_date[:-2] + ":" + new_end_date[-2:]

    body_request_1_Year_ = {
      "startDate": new_start_date,
      "endDate": new_end_date,
    }


    [df_injection_quantitiy_columns_, df_injection_quanitity_] = get_data_from_api(url_2_injection_quantitiy_, body_request_1_Year_)

    current_date = next_year

    write_dataframe_to_database(df_injection_quanitity_, table_name = f'INJECTION_QUANTITY_{startDate_formatted_}_{endDate_formatted_}', connection = conn_, cursor = cur_)

# delete Dataframe and columns
del df_injection_quantitiy_columns_, df_injection_quanitity_

INJECTION_QUANTITY_2021_01_01_2023_12_31 Table now in database, database not closed, please close later
INJECTION_QUANTITY_2021_01_01_2023_12_31 Table now in database, database not closed, please close later
INJECTION_QUANTITY_2021_01_01_2023_12_31 Table now in database, database not closed, please close later


## Forecast Load and Demand

**Forecast Load**

Spalten sind: Date, Hour, Load Forecast Plan (MWh)



In [ ]:
url_2_forecast_load_ = "/v1/consumption/data/load-estimation-plan"
[df_forecast_load_columns_, df_forecast_load_] = get_data_from_api(url_2_forecast_load_, body_request_)

write_dataframe_to_database(df_forecast_load_, table_name = f'FORECAST_LOAD_{startDate_formatted_}_{endDate_formatted_}', connection = conn_, cursor = cur_)

# delete Dataframe and columns
del df_forecast_load_columns_, df_forecast_load_

FORECAST_LOAD_2021_01_01_2023_12_31 Table now in database, database not closed, please close later


**Realtime Consumption**

Spalten sind: Date, Hour, Consumption Quantitiy (MWh)

In [ ]:
url_2_real_time_consumption_ = "/v1/consumption/data/realtime-consumption"

[df_real_time_consumption_columns_, df_real_time_consumption_] = get_data_from_api(url_2_real_time_consumption_, body_request_)

write_dataframe_to_database(df_real_time_consumption_, table_name = f'REAL_TIME_CONSUMPTION_{startDate_formatted_}_{endDate_formatted_}', connection = conn_, cursor = cur_)

# delete Dataframe and columns
del df_real_time_consumption_columns_, df_real_time_consumption_

REAL_TIME_CONSUMPTION_2021_01_01_2023_12_31 Table now in database, database not closed, please close later


## Failure Information Load

Hier bekommt man nur für einen Tag die Failure Informationen, die diesen Tag betreffen: was wir machen müssen ist, dass wir alle Tage zwischen Start und Enddatum nehmen müssen und für jeden tag die Abfrage laufen lassen. Dabei hängen wir dann die DataFrames aneinander (untereinander). Zum Schluss müssen wir noch alle Duplikate rauswerfen.

**Failure Planned**

Spalten sind: Town,
District Name,
Date,
Distribution Company,
Start Date - Time,
End Date - Time,
Cause of Cutting,
Regions (Neighborhoods),
Number of Affected Customers,
Hourly Quantity of Withdrawal (KWh)

In [ ]:
url_2_planned_failure_ = "/v1/consumption/data/planned-power-outage-info"

#cur_.execute(f"DROP TABLE IF EXISTS PLANNED_FAILURE_{startDate_formatted_}_{endDate_formatted_}")

# diese Abfrage muss dann die jweieeligen Daten beinhalten

startdate_datetime_ = datetime.strptime(startDate_, "%Y-%m-%dT%H:%M:%S%z")
endDate_datetime_ = datetime.strptime(endDate_, "%Y-%m-%dT%H:%M:%S%z")

next_day_datetime_ = startdate_datetime_

start_date = pd.Timestamp('2021-01-01', tz='Etc/GMT-3')
end_date = pd.Timestamp('2023-12-31', tz='Etc/GMT-3')

# Schleife, die die Daten ausgibt
current_date = start_date
while current_date <= end_date:

  new_start_date = current_date.strftime("%Y-%m-%dT%H:%M:%S%z")

  new_start_date = new_start_date[:-2] + ":" + new_start_date[-2:]

  body_request_failure_ = {
      "period": new_start_date,
  }

  [df_planned_failure_columns_, df_planned_failure_consumption_] = get_data_from_api(url_2_planned_failure_, body_request_failure_)
  try:
    write_dataframe_to_database(df_planned_failure_consumption_, table_name = f'PLANNED_FAILURE_{startDate_formatted_}_{endDate_formatted_}', connection = conn_, cursor = cur_, show_succ_msg = False)
  except:
    print(f'Für den {current_date} gibt es keinen Ausfall')
  current_date += pd.Timedelta(days=1)

  # delete Dataframe and columns
  del df_planned_failure_columns_, df_planned_failure_consumption_

2023-06-29 00:00:00+03:00
2023-06-30 00:00:00+03:00
2023-07-01 00:00:00+03:00
2023-07-02 00:00:00+03:00
2023-07-03 00:00:00+03:00
2023-07-04 00:00:00+03:00
2023-07-05 00:00:00+03:00
2023-07-06 00:00:00+03:00
2023-07-07 00:00:00+03:00
2023-07-08 00:00:00+03:00
2023-07-09 00:00:00+03:00
2023-07-10 00:00:00+03:00
2023-07-11 00:00:00+03:00
2023-07-12 00:00:00+03:00
2023-07-13 00:00:00+03:00
2023-07-14 00:00:00+03:00
2023-07-15 00:00:00+03:00
2023-07-16 00:00:00+03:00
2023-07-17 00:00:00+03:00
2023-07-18 00:00:00+03:00
2023-07-19 00:00:00+03:00
2023-07-20 00:00:00+03:00
2023-07-21 00:00:00+03:00
2023-07-22 00:00:00+03:00
2023-07-23 00:00:00+03:00
2023-07-24 00:00:00+03:00
2023-07-25 00:00:00+03:00
2023-07-26 00:00:00+03:00
2023-07-27 00:00:00+03:00
2023-07-28 00:00:00+03:00
2023-07-29 00:00:00+03:00
2023-07-30 00:00:00+03:00
2023-07-31 00:00:00+03:00
2023-08-01 00:00:00+03:00
2023-08-02 00:00:00+03:00
2023-08-03 00:00:00+03:00
2023-08-04 00:00:00+03:00
2023-08-05 00:00:00+03:00
2023-08-06 0

**Failure Unplanned**

Spalten sind: Town,
District Name,
Date,
Distribution Company,
Start Date - Time,
End Date - Time,
Cause of Cutting,
Regions (Neighborhoods),
Number of Affected Customers,
Hourly Quantity of Withdrawal (KWh)

In [ ]:
url_2_unplanned_failure_ = "/v1/consumption/data/unplanned-power-outage-info"

cur_.execute(f"DROP TABLE IF EXISTS UNPLANNED_FAILURE_{startDate_formatted_}_{endDate_formatted_}")

start_date = pd.Timestamp('2021-01-01', tz='Etc/GMT-3')
end_date = pd.Timestamp('2023-12-31', tz='Etc/GMT-3')

# Schleife, die die Daten ausgibt
current_date = start_date
while current_date <= end_date:
  print(current_date)
  new_start_date = current_date.strftime("%Y-%m-%dT%H:%M:%S%z")

  new_start_date = new_start_date[:-2] + ":" + new_start_date[-2:]

  body_request_failure_ = {
      "period": new_start_date,
  }
  current_date += pd.Timedelta(days=1)
  try:
    [df_unplanned_failure_columns_, df_unplanned_failure_consumption_] = get_data_from_api(url_2_unplanned_failure_, body_request_failure_)
  except:
    print(f' für den {current_date} gibt es kein unplanned failure')
  write_dataframe_to_database(df_unplanned_failure_consumption_, table_name = f'UNPLANNED_FAILURE_{startDate_formatted_}_{endDate_formatted_}', connection = conn_, cursor = cur_, show_succ_msg = False)

# delete Dataframe and columns
  del df_unplanned_failure_columns_, df_unplanned_failure_consumption_

2021-01-01 00:00:00+03:00
2021-01-02 00:00:00+03:00
2021-01-03 00:00:00+03:00
2021-01-04 00:00:00+03:00
2021-01-05 00:00:00+03:00
2021-01-06 00:00:00+03:00
2021-01-07 00:00:00+03:00
2021-01-08 00:00:00+03:00
2021-01-09 00:00:00+03:00
2021-01-10 00:00:00+03:00
2021-01-11 00:00:00+03:00
2021-01-12 00:00:00+03:00
2021-01-13 00:00:00+03:00
2021-01-14 00:00:00+03:00
2021-01-15 00:00:00+03:00
2021-01-16 00:00:00+03:00
2021-01-17 00:00:00+03:00
2021-01-18 00:00:00+03:00
2021-01-19 00:00:00+03:00
2021-01-20 00:00:00+03:00
2021-01-21 00:00:00+03:00
2021-01-22 00:00:00+03:00
2021-01-23 00:00:00+03:00
2021-01-24 00:00:00+03:00
2021-01-25 00:00:00+03:00
2021-01-26 00:00:00+03:00
2021-01-27 00:00:00+03:00
2021-01-28 00:00:00+03:00
2021-01-29 00:00:00+03:00
2021-01-30 00:00:00+03:00
2021-01-31 00:00:00+03:00
2021-02-01 00:00:00+03:00
2021-02-02 00:00:00+03:00
2021-02-03 00:00:00+03:00
2021-02-04 00:00:00+03:00
2021-02-05 00:00:00+03:00
2021-02-06 00:00:00+03:00
2021-02-07 00:00:00+03:00
2021-02-08 0

## WPP Forecast and Generation


Spalten sind: Hour,
Forecast (MW),
Generation (MW)

Folgendes muss beachtet werden: Daten liegen im 10 Minuten ratser vor, d.h. wir müssen sie aggregieren in **Stundenformat:** Hier müssen Daten von Stunde 01:00 zu Stunde 01-02 zählen, diesen Wert noch nicht in dem Trading Zeitraum von Stunde 00-01 haben


In [ ]:
from dateutil.relativedelta import relativedelta

url_2_wpp_generation_and_forecast_ = "/v1/renewables/data/res-generation-and-forecast"

startDate_datetime_ = datetime.strptime(startDate_, "%Y-%m-%dT%H:%M:%S%z")
endDate_datetime_ = datetime.strptime(endDate_, "%Y-%m-%dT%H:%M:%S%z")

current_date = startDate_datetime_

while current_date <= endDate_datetime_:
    next_date = current_date + relativedelta(months=3)
    new_start_date = current_date.strftime("%Y-%m-%dT%H:%M:%S%z")
    new_end_date = next_date.strftime("%Y-%m-%dT%H:%M:%S%z")

    new_start_date = new_start_date[:-2] + ":" + new_start_date[-2:]
    new_end_date = new_end_date[:-2] + ":" + new_end_date[-2:]

    body_request_3_month_ = {
      "startDate": new_start_date,
      "endDate": new_end_date,
    }

    [df_wpp_generation_and_forecast_columns_, df_wpp_generation_and_forecast_] = get_data_from_api(url_2_wpp_generation_and_forecast_, body_request_3_month_)

    current_date = next_date

    write_dataframe_to_database(df_wpp_generation_and_forecast_, table_name = f'WPP_GENERATION_AND_FORECAST_{startDate_formatted_}_{endDate_formatted_}', connection = conn_, cursor = cur_, show_succ_msg=False)

# delete Dataframe and columns
del df_wpp_generation_and_forecast_columns_, df_wpp_generation_and_forecast_

## Balance Market

**System's Direction**

Spalten sind: Date, Hour, System Direction (Energy Surplus, Balanced, Energy Deficit)


In [ ]:
url_2_system_direction_ = "/v1/markets/bpm/data/system-direction"

[df_system_direction_columns_, df_system_direction_] = get_data_from_api(url_2_system_direction_, body_request_)

write_dataframe_to_database(df_system_direction_, table_name = f'SYSTEM_DIRECTION_{startDate_formatted_}_{endDate_formatted_}', connection = conn_, cursor = cur_)

# delete Dataframe and columns
del df_system_direction_columns_, df_system_direction_

SYSTEM_DIRECTION_2021_01_01_2023_12_31 Table now in database, database not closed, please close later


**Up Regulation Instructions**

Spalten sind: Date, Hour,
Up Regulation Instructions (0) (MWh),
Up Regulation Instructions (1) (MWh),
Up Regulation Instructions (2) (MWh)

Hier beschreiben (0), (1), (2) die unterschiedlichen "Anlagetypen". Phase 0 sind schnell anschaltbare Anlagen die oft nur für einen kurzen Zeitraum hochgefahren werden, Phase 1 dann mit einem längeren Zeitraum aber auch mehr Power und Kosten und Phase 2 etc.

In [ ]:
url_2_up_regulation_ = "/v1/markets/bpm/data/order-summary-up"

[df_up_regulation_columns_, df_up_regulation_] = get_data_from_api(url_2_up_regulation_, body_request_)

write_dataframe_to_database(df_up_regulation_, table_name = f'UP_REGULATION_{startDate_formatted_}_{endDate_formatted_}', connection = conn_, cursor = cur_)

# delete Dataframe and columns
del df_up_regulation_columns_, df_up_regulation_

UP_REGULATION_2021_01_01_2023_12_31 Table now in database, database not closed, please close later


**Down Regulation Instructions**

Spalten sind: Date, Hour,
Down Regulation Instructions (0) (MWh),
Down Regulation Instructions (1) (MWh),
Down Regulation Instructions (2) (MWh)

In [ ]:
url_2_down_regulation_ = "/v1/markets/bpm/data/order-summary-down"

[df_down_regulation_columns_, df_down_regulation_] = get_data_from_api(url_2_down_regulation_, body_request_)

write_dataframe_to_database(df_down_regulation_, table_name = f'DOWN_REGULATION_{startDate_formatted_}_{endDate_formatted_}', connection = conn_, cursor = cur_)

# delete Dataframe and columns
del df_down_regulation_columns_, df_down_regulation_

DOWN_REGULATION_2021_01_01_2023_12_31 Table now in database, database not closed, please close later


## Power Plant Failure

In [ ]:
url_2_market_message_ = "/v1/markets/data/market-message-system"

cur_.execute(f"DROP TABLE IF EXISTS FAILURE_MESSAGE_{startDate_formatted_}_{endDate_formatted_}")

body_request_region_ = {
    "startDate": startDate_,
    "endDate": endDate_,
    "regionId" : 1
}

[df_failure_message_columns_, df_failure_message_df_] = get_data_from_api(url_2_market_message_, body_request_region_)

write_dataframe_to_database(df_failure_message_df_.drop(columns = ['faultDetails']), table_name = f'FAILURE_MESSAGE_{startDate_formatted_}_{endDate_formatted_}', connection = conn_, cursor = cur_)

# delete Dataframe and columns
#del df_failure_message_columns_, df_failure_message_df_

FAILURE_MESSAGE_2021_01_01_2023_12_31 Table now in database, database not closed, please close later


In [ ]:
df_failure_message_df_

,orgName,powerPlantName,uevcbName,caseStartDate,caseEndDate,operatorPower,capacityAtCaseTime,reason,id,uevcbId,faultDetails
0,ELEKTRİK ÜRETİM A.Ş.,ILISU BARAJI ve HES,ILISU BARAJI ve HES,2023-11-28T00:00:00+03:00,2023-11-28T21:59:00+03:00,1208.60,0,BOĞULMA VAKASI.,356973,3211210,"[{'hour': '2023-11-28T00:00:00+03:00', 'remain..."
1,ADAPAZARI ELEKTRİK ÜRETİM LİMİTED ŞİRKETİ,ADAPAZARI DGKÇS,ADAPAZARI DGKÇS,2022-09-30T00:00:00+03:00,2022-09-30T23:59:00+03:00,818.00,0,Buhar Türbini - IP Türbin Arızası,313679,3205381,"[{'hour': '2022-09-30T00:00:00+03:00', 'remain..."
2,SİLOPİ ELEKTRİK ÜRETİM A.Ş,SİLOPİ TES,SİLOPİ TES 154kV,2022-08-28T00:08:00+03:00,2022-08-28T23:59:00+03:00,135.00,0,1.Ünite türbin arızasından dolayı servis harici.,309896,3964,"[{'hour': '2022-08-28T00:00:00+03:00', 'remain..."
3,ELEKTRİK ÜRETİM A.Ş.,TEKİRDAĞ DGKÇS_B(Doğalgaz/Motorin),TEKİRDAĞ DGKÇS_B(Doğalgaz/Motorin),2021-04-15T10:40:00+03:00,2021-04-15T10:59:00+03:00,478.00,350,START ÇALIŞMASI,246078,3204399,"[{'hour': '2021-04-15T10:00:00+03:00', 'remain..."
4,YEŞİLYURT ENERJİ ELEKTRİK ÜRETİM A.Ş.,SAMSUN OSB DGKÇS,Samsun OSB DGKÇS,2021-10-08T00:00:00+03:00,2021-10-08T23:59:00+03:00,234.45,216,7 Nolu makina yağlama pompası arızası nedeniyl...,267586,335813,"[{'hour': '2021-10-08T00:00:00+03:00', 'remain..."
...,...,...,...,...,...,...,...,...,...,...,...
93336,AKSA GÖYNÜK ENERJİ ÜRETİM ANONİM ŞİRKETİ,BOLU-GÖYNÜK ENERJİ SANTRALİ,BOLU-GÖYNÜK ENERJİ SANTRALİ_1,2021-01-05T03:00:00+03:00,2021-01-05T04:59:00+03:00,135.00,83,kül sevk sistemi arızası,232458,3192481,"[{'hour': '2021-01-05T03:00:00+03:00', 'remain..."
93337,KÜÇÜK ENERJİ ÜRETİM VE TİCARET LİMİTED ŞİRKETİ,KÖPRÜBAŞI HES(KÜÇÜK ENR.),KÖPRÜBAŞI HES(KÜÇÜK ENR.),2022-10-05T00:05:00+03:00,2022-10-05T23:59:00+03:00,7.95,0,İletim kanalında göçük,314198,3194111,"[{'hour': '2022-10-05T00:00:00+03:00', 'remain..."
93338,GARET ENERJİ ÜRETİM VE TİCARET A.Ş.,SARES RES,SARES RES,2022-03-29T02:41:00+03:00,2022-03-29T12:59:00+03:00,27.50,25,1 TÜRBİN ARIZA,291735,26808,"[{'hour': '2022-03-29T02:00:00+03:00', 'remain..."
93339,ABK ENERJİ ELEKTRİK ÜRETİM A.Ş.,Söke-Çatalbük Rüzgar Elektrik Santrali,SÖKE ÇATALBÜK,2021-10-11T00:10:00+03:00,2021-10-11T23:59:00+03:00,30.00,26,Türbin ARızası,268007,81786,"[{'hour': '2021-10-11T00:00:00+03:00', 'remain..."


In [ ]:
conn_.commit()
conn_.close()

## IDM Transaction History



Hier werden jetzt die Trading Daten gezogen. Da das die einzelnen tatsächlich getätigten Trades sind, sind das ziemlich große Daten. Wir haben dafür eine nuee Datenbank angelegt, um eine bessere Übersicht zu haben.

In [ ]:
[conn_trading_IDM_, cur_trading_IDM_] = connect_to_database("original_data_trading_IDM_Transaction.db")

In [ ]:
url_2_idm_transaction_history_ = "/v1/markets/idm/data/transaction-history"

start_date = pd.Timestamp('2021-01-01', tz='Etc/GMT-3')
end_date = pd.Timestamp('2023-12-31', tz='Etc/GMT-3')

#cur_trading_IDM_.execute(f"DROP TABLE IF EXISTS IDM_TRANSACTION_HISTORY_{startDate_formatted_}_{endDate_formatted_}")

# Schleife, die die Daten ausgibt
current_start_date = start_date
while current_start_date <= end_date:
  new_end_date = current_start_date + pd.Timedelta(days=1)
  print(current_start_date)
  new_start_date = current_start_date.strftime("%Y-%m-%dT%H:%M:%S%z")
  new_end_date = new_end_date.strftime("%Y-%m-%dT%H:%M:%S%z")

  new_start_date = new_start_date[:-2] + ":" + new_start_date[-2:]
  new_end_date = new_end_date[:-2] + ":" + new_end_date[-2:]

  body_request_daily_ = {
      "startDate": new_start_date,
      "endDate": new_end_date,
  }
  current_start_date += pd.Timedelta(days=1)
  try:
    [df_idm_transaction_history_columns_, df_idm_transaction_history_] = get_data_from_api(url_2_idm_transaction_history_, body_request_daily_)
    df_idm_transaction_history_ = df_idm_transaction_history_
  except:
    print(f' für den {current_start_date} gibt es kein unplanned failure')
  write_dataframe_to_database(df_idm_transaction_history_, table_name = f'IDM_TRANSACTION_HISTORY_{startDate_formatted_}_{endDate_formatted_}', connection = conn_trading_IDM_, cursor = cur_trading_IDM_, show_succ_msg = False)

# delete Dataframe and columns
  del df_idm_transaction_history_columns_, df_idm_transaction_history_

2021-08-08 00:00:00+03:00
2021-08-09 00:00:00+03:00
2021-08-10 00:00:00+03:00
2021-08-11 00:00:00+03:00
2021-08-12 00:00:00+03:00
2021-08-13 00:00:00+03:00
2021-08-14 00:00:00+03:00
2021-08-15 00:00:00+03:00
2021-08-16 00:00:00+03:00
2021-08-17 00:00:00+03:00
2021-08-18 00:00:00+03:00
2021-08-19 00:00:00+03:00
2021-08-20 00:00:00+03:00
2021-08-21 00:00:00+03:00
2021-08-22 00:00:00+03:00
2021-08-23 00:00:00+03:00
2021-08-24 00:00:00+03:00
2021-08-25 00:00:00+03:00
2021-08-26 00:00:00+03:00
2021-08-27 00:00:00+03:00
2021-08-28 00:00:00+03:00
2021-08-29 00:00:00+03:00
2021-08-30 00:00:00+03:00
2021-08-31 00:00:00+03:00
2021-09-01 00:00:00+03:00
2021-09-02 00:00:00+03:00
2021-09-03 00:00:00+03:00
2021-09-04 00:00:00+03:00
2021-09-05 00:00:00+03:00
2021-09-06 00:00:00+03:00
2021-09-07 00:00:00+03:00
2021-09-08 00:00:00+03:00
2021-09-09 00:00:00+03:00
2021-09-10 00:00:00+03:00
2021-09-11 00:00:00+03:00
2021-09-12 00:00:00+03:00
2021-09-13 00:00:00+03:00
2021-09-14 00:00:00+03:00
2021-09-15 0

In [ ]:
conn_trading_IDM_.commit()
conn_trading_IDM_.close()